In [ ]:
import random
import heapq
import time
import tracemalloc

#ucs    
class Node:
    def __init__(self, state, cost):
        self.state = state
        self.cost = cost

    def __lt__(self, other):
        return self.cost < other.cost

def count_conflicts(state):
    conflicts = 0
    for i in range(len(state)):
        for j in range(i + 1, len(state)):
            if state[i] == state[j] or abs(state[i] - state[j]) == j - i:
                conflicts += 1
    return conflicts

def uniform_cost_search(n, max_steps=20000):
    # Initialize the starting state
    state = list(range(n))
    random.shuffle(state)
    cost = count_conflicts(state)
    node = Node(state, cost)

    # Initialize the priority queue
    queue = []
    heapq.heappush(queue, node)

    # Initialize the step counter
    steps = 0

    # Search loop
    while queue:
        # Check if the maximum number of steps has been reached
        if steps == max_steps:
            return None

        # Pop the node with the lowest cost
        node = heapq.heappop(queue)
        state = node.state

        # Check if the state is a goal state
        if count_conflicts(state) == 0:
            return state

        # Generate the successor states
        successors = []
        for i in range(n):
            for j in range(n):
                if state[i] != j:
                    successor = state[:]
                    successor[i] = j
                    cost = count_conflicts(state)
                    successors.append(Node(successor, cost))

        # Add the successor states to the queue
        for successor in successors:
            heapq.heappush(queue, successor)

        # Increment the step counter
        steps += 1

    # Return None if no solution is found
    return None

# A* search
def a_star_search(n):
    # Khởi tạo bàn cờ ngẫu nhiên
    board = [random.randint(0, n-1) for i in range(n)]
    start_node = tuple(board)

    # Hàm tạo các node kế tiếp và tính toán cost của chúng
    def get_successors(node):
        successors = []
        board = list(node)
        for col in range(n):
            for row in range(n):
                if board[col] != row:
                    new_board = list(board)
                    new_board[col] = row
                    new_node = tuple(new_board)
                    cost = count_conflicts(new_board) + heuristic(new_node)
                    successors.append((new_node, cost))
        return successors

    # Hàm tính toán heuristic cost cho một node
    def heuristic(node):
        board = list(node)
        return count_conflicts(board)

    # Hàm kiểm tra node là goal state hay không
    def is_goal(node):
        board = list(node)
        return count_conflicts(board) == 0

    # Tìm đường đi bằng thuật toán A* với min conflict
    frontier = [(heuristic(start_node), start_node)]
    explored = set()
    while frontier:
        _, node = heapq.heappop(frontier)
        if is_goal(node):
            return list(node)
        explored.add(node)
        for successor, cost in get_successors(node):
            if successor not in explored:
                heapq.heappush(frontier, (cost, successor))

    # Không tìm thấy giải pháp
    return None

#genetic algorithm with min conflict heuristic
def generate_population(n, N):
    """Generate an initial population of N n-queens solutions with n queens."""
    population = []
    for i in range(N):
        solution = list(range(n))
        random.shuffle(solution)
        population.append(solution)
    return population

def fitness(solution):
    """Compute the fitness of an n-queens solution."""
    conflicts = 0
    n = len(solution)
    for i in range(n):
        for j in range(i+1, n):
            if solution[i] == solution[j] or abs(solution[i] - solution[j]) == j - i:
                conflicts += 1
    return n*(n-1)//2 - conflicts

def tournament_selection(population, M, k):
    """Select M individuals from the population using tournament selection with k competitors."""
    selected = []
    for i in range(M):
        competitors = random.sample(population, k)
        winner = max(competitors, key=fitness)
        selected.append(winner)
    return selected

def crossover(parent1, parent2):
    """Generate a child solution by applying one-point crossover to two parent solutions."""
    n = len(parent1)
    c = random.randint(0, n-1)
    child = parent1[:c] + parent2[c:]
    return child

def mutation(solution):
    """Mutate a solution by randomly swapping two queens."""
    n = len(solution)
    i, j = random.sample(range(n), 2)
    solution[i], solution[j] = solution[j], solution[i]
    return solution

def genetic_algorithm(n, N, M, k, max_steps):
    """Solve the n-queens problem using a genetic algorithm."""
    population = generate_population(n, N)
    for i in range(max_steps):
        fittest = max(population, key=fitness)
        if fitness(fittest) == n*(n-1)//2:
            return fittest
        parents = tournament_selection(population, M, k)
        new_population = []
        while len(new_population) < N:
            parent1, parent2 = random.sample(parents, 2)
            child = crossover(parent1, parent2)
            child = mutation(child)
            new_population.append(child)
        population = new_population
    return None

# Print board
def print_board(board):
    n = len(board)
    for i in range(n):
        for j in range(n):
            if board[i] == j:
                print("Q ", end="")
            else:
                print("* ", end="")
        print()

def run_algorithms(n, algo_choice):
    if algo_choice == 1:
        # Uniform-cost search
        solution = uniform_cost_search(n)
    elif algo_choice == 2:
        # A* search
        solution = a_star_search(n)
    elif algo_choice == 3:
        # Genetic algorithm
        solution = genetic_algorithm(n, N=1000, M=500, k=500, max_steps=1000)
    else:
        print("Invalid algorithm choice")
        return
    return solution


if __name__ == "__main__":
    #choose size of board
    n = int(input("Enter the number of queens: "))
    #choose algorithm
    algorithm_choice = int(input("Choose the algorithm (1: Uniform-cost search, 2: A* with MIN-CONFLICT, 3: Genetic algorithm): "))

    # Start measuring time
    start_time = time.time()
    tracemalloc.start()
    mem_usage_before = tracemalloc.get_traced_memory()[0]

    solution = run_algorithms(n, algorithm_choice)

    mem_usage_after = tracemalloc.get_traced_memory()[0]
    tracemalloc.stop()
    # End measuring time and print elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time
    

    # Print the solution
    if solution is None:
        print("No solution found")
    else:
        print_board(solution)

    print("Elapsed time: {:.2f} ms".format(elapsed_time * 1000))
    print(f"Memory usage: {(mem_usage_after - mem_usage_before) / (1024 * 1024):.4f} MB")



